In [22]:
hidden_dim = None

dim = 4096
multiple_of = 4

hidden_dim = 4 * dim
hidden_dim = int(2 * hidden_dim / 3)
hidden_dim = multiple_of * ((hidden_dim + multiple_of - 10) // multiple_of)

print(hidden_dim)

10916


In [14]:
4 * int(2 * dim / 3)

10920

In [15]:
10924.0 / 4.0

2731.0